#### CSC 180-01 Intelligent Systems (Fall 2022)
##### Group: Jonathan Jakab,  Jacob Hallum, Mahdi Noureddine
##### California State University, Sacramento

# Project 2: Network Intrusion Detection System

### Used to allocate a specific amount of memory to GPU.
#### Note: Can not be reinitialized once the program is running. Chose VRAM amount before as MB
##### tf.config.LogicalDeviceConfiguration(memory_limit=10240) <----

In [1]:
# Used to test GPU support
import tensorflow as tf

#Checks if GPU is available and prints physical devices
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.list_physical_devices('GPU')

# Sets max GPU memory usage
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 10GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=10240)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Imports
Contains the imports for the applications. 

In [2]:
import os
import csv
import json
import shutil
import numpy as np
import pandas as pd

from collections.abc import Sequence

import matplotlib.pyplot as plt

from sklearn import datasets, metrics, svm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as sk_text
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

# Helpful Functions for Tensorflow (from Labs)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [3]:
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
    
    
# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l

    
# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

    
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.xlabel('business')
    plt.legend()
    plt.show()

    
# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low    

# Input/Output Setup
* ***Checks directories for Data input and output***
* ***Reads in data from the review file in the data directory***

In [4]:
# Get current working directory
current_directory = os.getcwd()

# Sets up input file locations for the data input.
dataPath = "./data/"
dataDirectory = os.path.join(current_directory, dataPath)

# Sets up locations for the data output location.
savePath = "./output/"
saveDirectory = os.path.join(current_directory, savePath)

if os.path.exists(saveDirectory):
    print('Directory Exists')
else:
    os.makedirs(saveDirectory)

Directory Exists


# Data Processing
## Read in reviews from network_intrusion_data.csv
### Columns in dataset
* duration: continuous.
* protocol_type: symbolic.
* service: symbolic.
* flag: symbolic.
* src_bytes: continuous.
* dst_bytes: continuous.
* land: symbolic.
* wrong_fragment: continuous.
* urgent: continuous.
* hot: continuous.
* num_failed_logins: continuous.
* logged_in: symbolic.
* num_compromised: continuous.
* root_shell: continuous.
* su_attempted: continuous.
* num_root: continuous.
* num_file_creations: continuous.
* num_shells: continuous.
* num_access_files: continuous.
* num_outbound_cmds: continuous.
* is_host_login: symbolic.
* is_guest_login: symbolic.
* count: continuous.
* srv_count: continuous.
* serror_rate: continuous.
* srv_serror_rate: continuous.
* rerror_rate: continuous.
* srv_rerror_rate: continuous.
* same_srv_rate: continuous.
* diff_srv_rate: continuous.
* srv_diff_host_rate: continuous.
* dst_host_count: continuous.
* dst_host_srv_count: continuous.
* dst_host_same_srv_rate: continuous.
* dst_host_diff_srv_rate: continuous.
* dst_host_same_src_port_rate: continuous.
* dst_host_srv_diff_host_rate: continuous.
* dst_host_serror_rate: continuous.
* dst_host_srv_serror_rate: continuous.
* dst_host_rerror_rate: continuous.
* dst_host_srv_rerror_rate: continuous.

### Attack Types:
* back,buffer_overflow,
* ftp_write,
* guess_passwd,
* imap,
* ipsweep,
* land,
* loadmodule,
* multihop,
* neptune,
* nmap,
* normal,
* perl,
* phf,
* pod,
* portsweep,
* rootkit,
* satan,
* smurf,
* spy,
* teardrop,
* warezclient,
* warezmaster

In [5]:
columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent',
           'hot', 'num_failed', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 
           'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',  'count', 'srv_count',
           'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 
           'srv_diff_srv_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
           'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
           'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'results']

intrusion_data = pd.read_csv(dataDirectory+'network_intrusion_data.csv', delimiter = ',', names=columns)
intrusion_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,results
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


## Check Records For Missing/Null Values 

In [6]:
intrusion_data.isnull().sum()

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed                     0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_s

## Drop Rows With Missing Values

In [7]:
intrusion_data.dropna(how='any', subset = columns)
intrusion_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,results
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


## Check For Duplicates

In [8]:
num_duplicates = intrusion_data.duplicated()
print("Number of Duplicates:", num_duplicates.sum())

Number of Duplicates: 348435


## Drop Any Duplicate Records

In [9]:
intrusion_data.drop_duplicates(inplace = True)
intrusion_data.reset_index(inplace = True)
intrusion_data.drop('index', axis = 1, inplace = True)
intrusion_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,results
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.00,0.00,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.00,0.00,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.00,0.00,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.00,0.00,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145581,0,tcp,http,SF,310,1881,0,0,0,0,...,255,1.0,0.0,0.01,0.05,0.00,0.01,0.0,0.0,normal.
145582,0,tcp,http,SF,282,2286,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.00,0.01,0.0,0.0,normal.
145583,0,tcp,http,SF,203,1200,0,0,0,0,...,255,1.0,0.0,0.06,0.05,0.06,0.01,0.0,0.0,normal.
145584,0,tcp,http,SF,291,1200,0,0,0,0,...,255,1.0,0.0,0.04,0.05,0.04,0.01,0.0,0.0,normal.


# 
# Data Prepping

## Encode Categorical Features
* Professors Helpful Functions state to use encode_text_dummy()

### Selecting Features
Using the documentation, we selected features felt provided key information on how the user got into the system.

* ***protocol_type:*** gives us the type of protocol used.
* ***service:*** shows the network service on the destination.
* ***flag:*** shows the status of the connection
* ***land:*** tells us if the connection was from/to the host port or not.
* ***su_attempted:*** Checks if su_root command was attempted. SU commands change the credentials of users to root or whatever they wish to set.
* ***root_shell:*** indicates if the user obtained a root shell.
* ***loggedin, is_host_login, is_guest_login:*** indicates how the user logged in.

### Warning: If this is run already, restart the kernal if any changes are made. Rerunning this again will cause issues.

In [10]:
encode_text_dummy(intrusion_data, 'protocol_type')
encode_text_dummy(intrusion_data, 'service')
encode_text_dummy(intrusion_data, 'flag')
encode_text_dummy(intrusion_data, 'land')
encode_text_dummy(intrusion_data, 'logged_in')
encode_text_dummy(intrusion_data, 'root_shell')
encode_text_dummy(intrusion_data, 'su_attempted')
encode_text_dummy(intrusion_data, 'is_host_login')
encode_text_dummy(intrusion_data, 'is_guest_login')

 ## Normalize Raw Values
 ### Get the length of the dataset
 #### Note: The length is to the end of the original dataset. So the results column 


In [11]:
dataset_length = intrusion_data.columns.get_loc('results')
print('The length of the dataset is: ', dataset_length)

The length of the dataset is:  32


### Normalize every Column

In [12]:
for i in range (dataset_length):
    encode_numeric_zscore(intrusion_data, intrusion_data.columns[i])

## Check Records For Missing/Null Values  After Normalization

In [13]:
#Dataframe has to many columns so we can not see which columns have NA?NULL values.
# intrusion_data.isnull().sum()

# Instead of showing all columns, we will only display the columns name if it has NA/NULL
missing_value_columns = intrusion_data.columns[intrusion_data.isna().any()].tolist()
missing_value_columns

['num_outbound_cmds']

## Drop Records For Missing/Null Values 

In [14]:
intrusion_data.drop(missing_value_columns, axis = 1, inplace = True)
intrusion_data

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed,num_compromised,num_root,num_file_creations,...,logged_in-0,logged_in-1,root_shell-0,root_shell-1,su_attempted-0,su_attempted-1,su_attempted-2,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.10785,-0.004293,0.042595,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
1,-0.10785,-0.004261,-0.039036,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
2,-0.10785,-0.004263,-0.025042,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
3,-0.10785,-0.004272,-0.025042,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
4,-0.10785,-0.004273,-0.013613,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145581,-0.10785,-0.004222,-0.016096,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145582,-0.10785,-0.004237,-0.009436,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145583,-0.10785,-0.004281,-0.027295,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145584,-0.10785,-0.004232,-0.027295,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0


## Convert to Binary Classification Problem
#### Note: Run At Your Own Risk. It Took ME 1.5 HOURS!!!!

In [15]:
intrusion_data.groupby('results')['results'].count()

results
back.                 968
buffer_overflow.       30
ftp_write.              8
guess_passwd.          53
imap.                  12
ipsweep.              651
land.                  19
loadmodule.             9
multihop.               7
neptune.            51820
nmap.                 158
normal.             87832
perl.                   3
phf.                    4
pod.                  206
portsweep.            416
rootkit.               10
satan.                906
smurf.                641
spy.                    2
teardrop.             918
warezclient.          893
warezmaster.           20
Name: results, dtype: int64

In [16]:
for i in range(intrusion_data.shape[0]):
    if('normal' not in intrusion_data['results'][i]):
        intrusion_data['results'][i] = 'attack'
        print(i)

C:\Users\Jonny Apple Seed\AppData\Local\Temp\ipykernel_39236\2895196592.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intrusion_data['results'][i] = 'attack'


744
745
4042
4106
7573
7574
7765
7766
7767
7768
7769
7770
7771
7772
7773
7774
7775
7776
7777
7778
7779
7780
7781
7782
7783
7784
7785
7786
7787
7788
7789
7790
7791
7792
7793
7794
7795
7796
7797
7798
7799
7800
7801
7802
7803
7804
7805
7806
7807
7808
7809
7810
7811
7812
7813
7814
7815
7816
7817
7818
7819
7820
7821
7822
7823
7824
7825
7826
7827
7828
7829
7830
7831
7832
7833
7834
7835
12008
12093
12094
12095
12096
12097
12098
12099
12100
12101
12102
12103
12104
12105
12106
12107
12108
12109
12110
12111
12112
15575
15576
15577
15578
15579
15580
15581
15582
15583
15584
15585
15586
15587
15588
15589
15590
15591
15592
15593
15594
15595
15596
15597
15598
15599
15600
15601
15602
15603
15604
15605
15606
15607
15608
15609
15610
15611
15612
15613
15614
15615
15616
15617
15618
15619
15620
15621
15622
15623
15624
15625
15626
15627
15628
15629
15630
15631
15632
15633
15634
15635
15636
15637
15638
15639
15640
15641
15642
15643
15644
15645
15646
15647
15648
15649
15650
15651
15652
15653
15654
15655
15656

43542
43543
43544
43545
43546
43547
43548
43549
43550
43551
43552
43553
43554
43555
43556
43557
43558
43559
43560
43561
43562
43563
43564
43565
43566
43567
43568
43569
43570
43571
43572
43573
43574
43575
43576
43577
43578
43579
43580
43581
43582
43583
43584
43585
43586
43587
43588
43589
43590
43591
43592
43593
43594
43595
43596
43597
43598
43599
43600
43601
43602
43603
43604
43605
43606
43607
43608
43609
43610
43611
43612
43613
43614
43615
43616
43617
43618
43619
43620
43621
43622
43623
43624
43625
43626
43627
43628
43629
43630
43631
43632
43633
43634
43635
43636
43637
43638
43639
43640
43641
43642
43643
43644
43645
43646
43647
43648
43649
43650
43651
43652
43653
43654
43655
43656
43657
43658
43659
43660
43661
43662
43663
43664
43665
43666
43667
43668
43669
43670
43671
43672
43673
43674
43675
43676
43677
43678
43679
43680
43681
43682
43683
43684
43685
43686
43687
43688
43689
43690
43691
43692
43693
43694
43695
43696
43697
43698
43699
43700
43701
43702
43703
43704
43705
43706
43707
4370

In [17]:
intrusion_data.groupby('results')['results'].count()

results
attack     57754
normal.    87832
Name: results, dtype: int64

In [18]:
binary_intrusion_data = encode_text_index(intrusion_data, 'results')
binary_intrusion_data

array(['attack', 'normal.'], dtype=object)

## Save prepped data to file
### This will make it easier for testing purposes

In [22]:
intrusion_data.to_csv(saveDirectory + 'results.csv', index=False)
intrusion_data

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed,num_compromised,num_root,num_file_creations,...,logged_in-0,logged_in-1,root_shell-0,root_shell-1,su_attempted-0,su_attempted-1,su_attempted-2,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.10785,-0.004293,0.042595,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
1,-0.10785,-0.004261,-0.039036,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
2,-0.10785,-0.004263,-0.025042,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
3,-0.10785,-0.004272,-0.025042,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
4,-0.10785,-0.004273,-0.013613,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145581,-0.10785,-0.004222,-0.016096,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145582,-0.10785,-0.004237,-0.009436,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145583,-0.10785,-0.004281,-0.027295,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145584,-0.10785,-0.004232,-0.027295,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0


## Load prepped data to file
### This will make it easier for testing purposes

In [23]:
ready_for_train = pd.read_csv(saveDirectory+'results.csv', delimiter = ',', encoding = "utf-8")
ready_for_train

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed,num_compromised,num_root,num_file_creations,...,logged_in-0,logged_in-1,root_shell-0,root_shell-1,su_attempted-0,su_attempted-1,su_attempted-2,is_host_login-0,is_guest_login-0,is_guest_login-1
0,-0.10785,-0.004293,0.042595,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
1,-0.10785,-0.004261,-0.039036,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
2,-0.10785,-0.004263,-0.025042,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
3,-0.10785,-0.004272,-0.025042,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
4,-0.10785,-0.004273,-0.013613,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145581,-0.10785,-0.004222,-0.016096,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145582,-0.10785,-0.004237,-0.009436,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145583,-0.10785,-0.004281,-0.027295,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0
145584,-0.10785,-0.004232,-0.027295,-0.084394,-0.004737,-0.070209,-0.018022,-0.007905,-0.01039,-0.020694,...,0,1,1,0,1,0,0,1,1,0


# Split data into Training and Test Data.

### Alterable Values
* ***Test_Size:***
* ***random_state:***
* ***Dense:***
* ***Epochs: number of epochs***
### Note: Im not what are the best values for these.

In [ ]:


x,y = to_xy(ready_to_train,"stars")

# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# Requirements
* Split data to training and test. Use training data to train your models and evaluate the model quality using test data
* You must use EarlyStopping when training neural networks using Tensorflow.
* Tuning the following hyperparameters when training neural networks using Tensorflow to record how they affect performance in your report. Save all the models you have tried as a proof in your notebook and tabulate your findings in your report.
    * Activation: relu, sigmoid, tanh
    * Layers and neuron counts
    * Optimizer: adam and sgd
    * Kernel number and kernel size (for CNN only)

### Encoding (Connection = 0, Attacks = 1)

# Model Testing

## Fully-Connected Neural Networks

## Convolutional Neural Networks (CNN)
For the convolutional layer, We must specify the following hyper-parameters :

* Number of filters
* Filter Size
* Stride
* Padding
* Activation Function/Non-Linearity


A filter is a square-shaped object that scans over the image. The more filters that we give to a convolutional layer, the more features it can detect.


## Max Pooling Layers

Max-pool layers downsample the data.  This technique can avoid overfitting.

*** Typically, you can always place a max-pool layer immediately following convolutional layer.***



# Example of Training CNNs with Tensorflow

The following sections describe how to use TensorFlow/Keras with CNNs.

* Hint: For CNN, find a way to view each connection as an image. Please refer to our lab tutorial on CNN for handling data types other than images. You may use either Conv2D or Conv1D.

## Comparing Recall, Precision, and F1-Score.

# Plot Confusion Matrix for Each Model

## Plotting Confusion Matrix for Fully-Connected Neural Network

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, pred)
print(cm)


print('Plotting confusion matrix')

plt.figure()
plot_confusion_matrix(cm, diagnosis)
plt.show()

print(classification_report(y_true, pred))

## Plotting ROC Curves

In [ ]:
pred = model.predict(x_test)
pred = pred[:,1] # Only positive class (M)
plot_roc(pred,y_true)

## Plotting Confusion Matrix for Convolutional Neural Network

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(y_true, pred)
print(cm)


print('Plotting confusion matrix')

plt.figure()
plot_confusion_matrix(cm, diagnosis)
plt.show()

print(classification_report(y_true, pred))

## Plotting ROC Curves

In [ ]:
pred = model.predict(x_test)
pred = pred[:,1] # Only positive class (M)
plot_roc(pred,y_true)

# Testing Neural Network Models (Relu + Adam + 4 Hidden Layers, 20, 10, 10, 5 Neurons)
1. ***relu_adam_4_model.add(Dense(20, input_dim=x.shape[1], activation='relu'))   # Hidden Layer 1, 20 Neurons***
2. ***relu_adam_4_model.add(Dense(10, activation='relu'))   # Hidden Layer 2, 10 Neurons***
3. ***relu_adam_4_model.add(Dense(10, activation='relu'))   # Hidden Layer 3, 10 Neurons***
4. ***relu_adam_4_model.add(Dense(5, activation='relu'))   # Hidden Layer 4, 5 Neurons***
5. ***relu_adam_4_model.add(Dense(1))   # Output***

Note: The number of Dense() functions not including Dense(1) is the number of hidden layers

In [ ]:
# MAKE SURE TO CHANGE THE FILE NAME WITH EACH ONE OR YOU'LL OVERWRITE DATA
checkpointer = ModelCheckpoint(filepath="dnn/best_weights_ReluAdam4.hdf5", verbose=0, save_best_only=True) # save best model

relu_adam_4_model = Sequential()
relu_adam_4_model.add(Dense(20, input_dim=x.shape[1], activation='relu'))
relu_adam_4_model.add(Dense(10, activation='relu'))
relu_adam_4_model.add(Dense(10, activation='relu'))
relu_adam_4_model.add(Dense(5, activation='relu'))
relu_adam_4_model.add(Dense(1))

relu_adam_4_model.compile(loss='mean_squared_error', optimizer='adam') 

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

relu_adam_4_model.fit(x_train,y_train, validation_data=(x_test,y_test), callbacks=[monitor,checkpointer],verbose=2,epochs=600)

# MAKE SURE TO CHANGE THE FILE NAME WITH EACH ONE OR YOU'LL GET BAD DATA.
relu_adam_4_model.load_weights('dnn/best_weights_ReluAdam4.hdf5') # load weights from best model

## RMSE Predictions for ALL models

In [ ]:
# Predict Relu + Adam + 4 Hidden Layers
relu_adam4_pred = relu_adam_4_model.predict(x_test)
relu_adam4_model_rmse = np.sqrt(metrics.mean_squared_error(relu_adam4_pred, y_test))
print(f'Relu Adam Model w/ 4 Hidden Layers RMSE: {relu_adam4_model_rmse}')

# Selects the Best Model from the above
# best_model_rmse = min([relu_adam2_model_rmse, relu_adam4_model_rmse, relu_sgd_model_rmse, sigmoid_adam_model_rmse, sigmoid_sgd_model_rmse, tanh_adam_model_rmse, tanh_sgd_model_rmse])
# print('\nBest Model RMSE:', best_model_rmse)

## Plot Linear Regression Chart
Regression lift chart with sorting on expected output (y_test)

In [ ]:
# Plot the chart for the best model. NOTE: best model must be manually entered.
bestModel = sigmoid_adam_pred
chart_regression(bestModel.flatten(),y_test, sort=True)

### Regression lift chart without sorting on expected output (y_test)

In [ ]:
chart_regression(bestModel.flatten(),y_test, sort=False)

# Selecting 3-5 arbitrary businesses to predict the outcome.

In [ ]:
print(len(sigmoid_adam_pred))
sigmoid_adam_pred

In [ ]:
# Split into train/test.
x_train, x_test, y_train, y_test = train_test_split(df_ready_for_sklearn[['business_id', 'name', 'categories']], df_ready_for_sklearn['stars'], test_size=0.20, random_state=42)

df_best_model_prediction = pd.DataFrame(sigmoid_adam_pred, columns = ['Prediction'])
df_x_test = pd.DataFrame(x_test).sort_index().reset_index()
df_y_test = y_test.reset_index()

result = pd.concat([df_x_test, df_y_test, df_best_model_prediction], axis = 1)

#Select 3-5 arbitrary businesses.
df_select = pd.DataFrame(result.loc[[0, 5, 10, 20, 40]])
df_select